词向量，顾名思义，词的特征向量。之前我们用one-hot表示，但它并不是一个好的选择。因为one-hot词向量无法表达不同词之间的相似度。例如，任何一对词的one-hot向量的余弦相似度都为0。所以才有了word2vec和GloVe等各种各样的词向量。

## 1. word2vec

2013年，Google团队发表了[word2vec](https://code.google.com/archive/p/word2vec/)工具。word2vec工具主要包含两个模型：跳字模型（skip-gram）和连续词袋模型（continuous bag of words，简称CBOW），以及两种高效训练的方法：负采样（negative sampling）和层序softmax（hierarchical softmax）。值得一提的是，word2vec词向量可以较好地表达不同词之间的相似和类比关系。

## 2. 模型

假设词典大小为|V|，我们将词典中的每个词与从0到|V|−1的整数一一对应：词典索引集V={0,1,…,|V|−1}。一个词在该词典中所对应的整数称为词的索引。对于词典中一个索引为i的词，它在作为中心词和背景词时的向量表示分别是$v_i和u_i$。而词典中所有词的这两种向量正是跳字模型所要学习的模型参数。

so,怎么学习得到这样的词向量呢？我们需要有个模型，有损失函数，有目标函数。

#### 2.１ skip-gram 跳字模型

在跳字模型中，我们用一个词来预测它在文本序列周围的词。例如，给定文本序列”the”, “man”, “hit”, “his”, 和”son”，跳字模型所关心的是，给定”hit”，生成它邻近词“the”, “man”, “his”, 和”son”的概率。在这个例子中，”hit”叫中心词，“the”, “man”, “his”, 和”son”叫背景词。由于”hit”只生成与它距离不超过2的背景词，该时间窗口的大小为2。

给定一个长度为T的文本序列中，t时刻的词为$w^{(t)}$。当时间窗口大小为m时，跳字模型需要最大化给定任一中心词生成背景词的概率：

$$\prod_{t=1}^T\prod_{-m\le j\le m,j\neq0}P(w^{t+j}|w^{(t)})$$

>极大似然估计～中心词为$w^{(t)}$时，窗口大小为m时，生成对应背景词概率的乘积，然后取对数。似然估计的原理就是，既然你出现了，那么就假设你的概率最大！

上述公式可以等价于以下**损失函数**：
$$-\frac{1}{T}\sum_{t=1}^T\sum_{-m\le j\le m,j\neq0}logP(w^{t+j}|w^{(t)})\quad(1)$$

有了损失函数，怎么和词向量联系起来呢？用v和u分别代表中心词和背景词的向量。

>在Glove里面，共现概率＝向量内积的指数形式：

$$P_{ik}=e^{v_i^T\tilde v_k} = \dfrac{x_{ik}}{x_i}$$

$$v_i^T\tilde v_k = log(x_{ik})-log(x_i)$$

在word2vec里面，使用的softmax函数。给定中心词wc在词典中索引为c，背景词wo在词典中索引为o，损失函数中的中心词生成背景词的概率可以使用softmax函数定义为:

$$P(w_0|w_c)=\dfrac{exp(u_o^Tv_c)}{\sum_{i\in V}exp(u_i^Tv_c)}\quad(2)$$

对比Glove和word2vec可以看出，word2vec的对于$P(w_o|w_c)$的定义更严格，更符合我们所想的原理。按理来说word2vec训练的词向量效果应该会更好吧～～但是分母的计算太过于复杂了。这也是为啥后来的人又改进发明了Glove这个东西～

把(2)式带入(1)式可得用词嵌入向量表示的损失函数。当序列长度T较大时，我们通常随机采样一个较小的子序列来计算损失函数并使用随机梯度下降优化该损失函数。通过微分，我们可以计算出上式生成概率的对数关于中心词向量$v_c$的梯度为：

$$\dfrac{\partial P(w_0|w_c)}{\partial v_c} = u_o - \sum_{j\in V}\dfrac{u_jexp(u_j^Tv_c)}{\sum_{i\in V}exp(u_i^Tv_c)} $$

等价于：$$\dfrac{\partial P(w_0|w_c)}{\partial v_c} = u_o - \sum_{j\in V}P(w_j|w_c)u_j$$

通过上面计算得到梯度后，我们可以使用随机梯度下降来不断迭代模型参数$v_c$。其他模型参数$u_o$的迭代方式同理可得。最终，对于词典中的任一索引为i的词，我们均得到该词作为中心词和背景词的两组词向量$v_i和u_i$。

#### 2.2 连续词袋模型(continuous bag of words, CBOW)

与skip-gram不同的是，CBOW是用背景词来预测生成该中心词的概率。同样的，给定一个长度为T的文本序列中，t时刻的词为$w^{(t)}$。当时间窗口大小为m时，连续词袋模型需要最大化由背景词生成任一中心词的概率：

$$\prod_{t=1}^{T}P(w^{(t)}|w^{(t-m)},...,w^{(t-1)},w^{(t+1)},...,w^{(t+m)})$$

极大似然估计～

其等价于一下损失函数：
$$-\sum_{t=1}^{T}logP(w^{(t)}|w^{(t-m)},...,w^{(t-1)},w^{(t+1)},...,w^{(t+m)})$$

我们需要学习得到的模型参数是词嵌入向量。为了将模型参数植入损失函数，就要用模型参数，即用词嵌入向量来表达损失函数，在这里也就是表达损失函数中的背景词$w_o$生成中心词$w_c$的概率。与skip-grams不同的是，背景词向量用v表示，中心词用u表示。

$$P(w_c|w_{01},...,w_{o2m}) = \dfrac{exp[u_c^T(v_{o1}+...+v_{o2m})/(2m)]}{\sum_{i\in V}exp[u_i^T(v_{o1}+...+v_{o2m})/(2m)]}$$

通过微分，可以算出上式中生成概率的对数关于任一背景词$V_{oi}(i=1,...,2m)$的梯度为：

$$\dfrac{\partial P(w_c|w_{01},...,w_{o2m})}{\partial v_{oi} } = \dfrac{1}{2m}(u_c - \sum_{j\in V}\dfrac{exp(u_j^Tv_c)}{\sum_{i\in V}exp(u_i^Tv_c)}u_j)$$

注意这里与skip-grams不同，因为是用背景词生成中心词，故而是对背景词一次求微分。

等价于：
$$\dfrac{\partial P(w_c|w_{01},...,w_{o2m})}{\partial u_{oi} } = \dfrac{1}{2m}(u_c - \sum_{j\in V}P(w_j|w_c)u_j)$$

通过上面计算得到梯度后，我们可以使用随机梯度下降来不断迭代各个模型参数$v_{oi}(i=1,…,2m)$。其他模型参数$u_c$的迭代方式同理可得。最终，对于词典中的任一索引为i的词，我们均得到该词作为背景词和中心词的两组词向量$v_i和u_i$。

### 3. 近似训练法

>**总结**：上述模型已确定，要学习的参数模型有了～其表达的损失函数有了～训练方法，也就是随机梯度下降也有了～回顾一下整个训练过程：

>以skip-grams为例：
- 从语料库中，选择一个中心词$w_c$,然后采用极大似然估计原理，求出生成背景词$w_o$的概率
- 从生成概率最大化$P(w_0|w_c)$，求出损失函数
- 使用需要学习得到的参数模型，也就是词向量来表达损失函数中的生成概率。
- 求出损失函数对中心词向量的梯度，然后用SGD来进行训练。
- **需要注意的是:**在语料库中，中心词的选择问题，是遍历词典中所有的词吗？还是语料库中均匀且随机的选择？
- 由中心词生成背景词使用的softmax函数的计算量$\sum_{j\in V}\dfrac{exp(u_j^Tv_c)}{\sum_{i\in V}exp(u_i^Tv_c)}u_j$太大。其也会出现在梯度中。



但梯度的计算开销与词典大小Ｖ相关，当词典较大时，$\sum_{j\in V}\dfrac{exp(u_j^Tv_c)}{\sum_{i\in V}exp(u_i^Tv_c)}u_j$的计算开销太大了。因此，我们采用近似的方法来计算这些梯度。近似训练法包括负采样和层序softmax.

来算一下，如果词典V有10000个词，那么训练一个词典所需的计算量$10000^{10000}$太特么大了

#### 3.１　负采样

[Mikolov et.al.,2013. Distributed representation of words and phrases and their compositionality]

以跳字模型来讨论负采样。

词典Ｖ大小之所以会在目标函数中出现，是因为中心词$wc生成背景词wo的概率P(w_o∣w_c)$使用了softmax，而softmax正是考虑了背景词可能是词典中的任一词，并体现在softmax的分母上。

词典V大小之所以会在目标函数中出现，是因为中心词wc生成背景词wo的概率ℙ(wo∣wc)使用了softmax，而softmax正是考虑了背景词可能是词典中的任一词，并体现在softmax的分母上。

我们不妨换个角度，假设中心词wc生成背景词wo由以下相互独立事件联合组成来近似

- 中心词wc和背景词wo同时出现在该训练数据窗口
- 中心词wc和第1个噪声词w1不同时出现在该训练数据窗口（噪声词w1按噪声词分布ℙ(w)随机生成，假设一定和wc不同时出现在该训练数据窗口）
- …
- 中心词wc和第K个噪声词wK不同时出现在该训练数据窗口（噪声词wK按噪声词分布ℙ(w)随机生成，假设一定和wc不同时出现在该训练数据窗口）


>简单理解就是：本来我们在求生成概率，中心词$w_c$生成背景词$w_o$时，是考虑了整个词典中的所有的词作为背景词的可能。故而分母中出现词典大小。

>负采样呢？就是在训练窗口，确确实实出现了的，我们考虑这个背景词，其他没有出现在这个训练窗口的词，我们叫它们噪声词，这些个噪声词，我们从词典中选择Ｋ个。**噪声词的选择以及Ｋ值的大小选择问题先留着**

>这里所谓的噪声词就是负样本（negative sampling）吧～

**又要注意了**，这里使用sigmoid()函数定义了一个概率：
$$P(D=1|w_o,w_c) = \sigma(u_o^Tv_c)\quad(3)$$　

那么，中心词$w_c$生成背景词$w_o$的概率的对数可以近似为：
$$logP(w_o|w_c) = log[P(D=1|w_o,w_c)]\prod_{k=1,w_k~P(w)}^KP(D=0|w_k,w_c)\quad(4)$$

>这里是在训练窗口中出现的背景词和负采样Ｋ个噪声词的联合概率。对数似然估计最大！！！

>其中噪声词，也就是负样本的选择，与噪声词的分布P(w)相关。

将(3)带入(4)可得：
$$logP(w_o|w_c) = log\dfrac{1}{1+exp(-u_o^Tv_c)}＋\sum_{k=1,w_k~P(w)}^Klog\dfrac{1}{1+exp(-u_k^Tv_c)}$$
似然估计告诉我们，要让这个概率最大～

则取其相反数，损失函数：
$$－logP(w_o|w_c) = －log\dfrac{1}{1+exp(-u_o^Tv_c)}-\sum_{k=1,w_k~P(w)}^Klog\dfrac{1}{１＋exp(u_k^Tv_c)}$$

**So, 损失函数计算复杂度从O(|V|)变成了O(K)**

同理，连续词袋模型也可以用负采样～　

公式看原文链接吧..[gluon－词向量](https://zh.gluon.ai/chapter_natural-language-processing/word2vec.html)

#### 3.2 层序softmax

至始至终词向量之的训练之所以复杂，就是因为在中心词生成背景词的生成概率使用了softmax函数，从而引入了整个词典。计算复杂度为$O(|V|)$

层序softmax利用了二叉树。树的每个叶子节点代表着词典Ｖ中的每个词。每个词wi相应的词向量为vi。我们以下图为例，来描述层序softmax的工作机制。
<img src="https://zh.gluon.ai/_images/hierarchical_softmax.svg">

假设L(w)为从二叉树的根到代表词w的叶子节点的路径上的节点数,节点设为n(w,i),节点向量为${u_{n(w,i)}}$.则skip-grams和CBOW中所需要计算的任意词$w_i$生成w的概率为：

$$P(w|w_i) = \prod_{j=1}^{L(w)-1}\sigma([n(w,j+1)==leftChild(n(w,j))]\cdot u_{n(w,j)}^Tv_j)\quad(5)$$

其中[　]的部分，如果x为真，[x]=1;反之[x]=-1.这里就是判断从根节点到叶子节点这条路径上，子节点是否是左儿子。

由于$\sigma(x)+\sigma(-x)=1$,则$w_i$生成词典中任何词的概率之和为１：
$$\sum_{j=1}^VP(w_j|w_i) = 1$$

让我们计算wi生成w3的概率，由于在二叉树中由根到w3的路径上需要向左、向右、再向左地遍历，我们得到:
$$P(w_3|w_i) = \sigma(u_{n(w_3,1)}^Tv_i)\sigma(-u_{n(w_3,2)}^Tv_i)\sigma(u_{n(w_3,3)}^Tv_i)$$


我们可以使用随机梯度下降在跳字模型和连续词袋模型中不断迭代计算字典中所有词向量v和非叶子节点的向量u。每次迭代的计算开销由$O(|V|)$降为二叉树的高度$O(log|V|)$。

很神奇。。只能说作者脑洞好大。。抛开数学上的推导，完全无法理解为何。。。

然后将(5)式新的生成概率带入到损失函数中取，进行迭代训练。其中的模型参数不仅包括所有的词向量v，还包括非叶子节点的向量u。